In [1]:
import pandas as pd

df = pd.read_csv("/content/sample_data/dataset1.csv")
df.head(5)

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern


In [2]:
X = df['text']
y = df['label']

In [3]:
!pip install textattack


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 

In [4]:
from textattack.augmentation import CharSwapAugmenter
charswap_aug = CharSwapAugmenter()
df['augmented_text'] = df['text'].apply(lambda x: charswap_aug.augment(x) if isinstance(x, str) else x)

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.867 seconds.
DEBUG:jieba:Loading m

In [5]:
df_original = df.copy()
df['text'] = df['augmented_text']
result_df = pd.concat([df_original, df], ignore_index=True)
result_df.drop(columns=['augmented_text'], inplace=True)
result_df

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency
1,158,Pillowcases & Shams,0,Not Dark Pattern
2,108,Write a review,0,Not Dark Pattern
3,1425,"To start your return, simply click on the foll...",0,Not Dark Pattern
4,1658,newsletter signup (privacy policy),0,Not Dark Pattern
...,...,...,...,...
4707,1304,[ lBra Panty Set ],0,Not Dark Pattern
4708,1038,[DressesFootearHatsHoodies & SweatshirtsJersey...,0,Not Dark Pattern
4709,1081,[nOly 4 left in stock],1,Scarcity
4710,767,[Trendinqg Players],0,Not Dark Pattern


In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [8]:
result_df['text'] = result_df['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [9]:
result_df['text'] = result_df['text'].apply(preprocess)

In [10]:
result_df

,page_id,text,label,Pattern Category
0,1012,FLASH SALE | LIMITED TIME Shop,1,Urgency
1,158,pillowcase Shams,0,Not Dark Pattern
2,108,write review,0,Not Dark Pattern
3,1425,start return simply click follow button add re...,0,Not Dark Pattern
4,1658,newsletter signup privacy policy,0,Not Dark Pattern
...,...,...,...,...
4707,1304,lBra Panty Set,0,Not Dark Pattern
4708,1038,dressesfootearhatshoodie sweatshirtsjerseyspan...,0,Not Dark Pattern
4709,1081,nOly 4 leave stock,1,Scarcity
4710,767,Trendinqg Players,0,Not Dark Pattern


In [11]:
X = result_df['text']
y = result_df['label']


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize and fit the vectorizer to your training data
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

# Initialize and train your classifier
clf = RandomForestClassifier()
clf.fit(X_train_vectorized, y_train)

sample_text = "3 left"
# Transform the sample text using the trained vectorizer
sample_text_vectorized = vectorizer.transform([sample_text])

# Make a prediction
prediction = clf.predict(sample_text_vectorized)

# Output the prediction
if prediction == 1:
    print("The sample text is classified as a dark pattern.")
else:
    print("The sample text is not classified as a dark pattern.")

The sample text is classified as a dark pattern.


In [23]:
import joblib
joblib.dump(clf,'rf2.joblib')

['rf2.joblib']